Optimizing Parameters

In [1]:
#Prerequisite code

import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

In [2]:
# Hyperparameters : parameters you control
learning_rate = 1e-3    #how much to update parameters at each batch
batch_size = 64
epochs = 5

In [7]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()   # bc/ gradients by default add up; to prevent double counting.
        loss.backward() # deposits gradients
        optimizer.step()    # adjust parameters with gradients

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

# iterate over test dataset to check model performance
def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [8]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 0.560117  [    0/60000]
loss: 0.662901  [ 6400/60000]
loss: 0.456154  [12800/60000]
loss: 0.693722  [19200/60000]
loss: 0.623565  [25600/60000]
loss: 0.602588  [32000/60000]
loss: 0.645364  [38400/60000]
loss: 0.697285  [44800/60000]
loss: 0.682522  [51200/60000]
loss: 0.602727  [57600/60000]
Test Error: 
 Accuracy: 78.6%, Avg loss: 0.614420 

Epoch 2
-------------------------------
loss: 0.545888  [    0/60000]
loss: 0.649055  [ 6400/60000]
loss: 0.445643  [12800/60000]
loss: 0.684717  [19200/60000]
loss: 0.615833  [25600/60000]
loss: 0.595891  [32000/60000]
loss: 0.631636  [38400/60000]
loss: 0.692475  [44800/60000]
loss: 0.676177  [51200/60000]
loss: 0.591635  [57600/60000]
Test Error: 
 Accuracy: 79.1%, Avg loss: 0.604793 

Epoch 3
-------------------------------
loss: 0.532718  [    0/60000]
loss: 0.636428  [ 6400/60000]
loss: 0.436007  [12800/60000]
loss: 0.676327  [19200/60000]
loss: 0.608580  [25600/60000]
loss: 0.589748  [32000/600